## Exploratory Data Analysis

In [1]:
import gc
import pandas as pd
import numpy as np
import lightgbm as lgb
from operator import xor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from ml_metrics import rmsle
%matplotlib inline

le = preprocessing.LabelEncoder()
lb = preprocessing.LabelBinarizer()

air = {
    "reserve": pd.read_csv("data/air/air_reserve.csv", parse_dates=["visit_datetime", "reserve_datetime"]),
    "store_info": pd.read_csv("data/air/air_store_info.csv"),
    "visit_data": pd.read_csv("data/air/air_visit_data.csv", parse_dates=["visit_date"])
}

hpg = {
    "reserve": pd.read_csv("data/hpg/hpg_reserve.csv", parse_dates=["visit_datetime", "reserve_datetime"]),
    "store_info": pd.read_csv("data/hpg/hpg_store_info.csv")
}

date_info = pd.read_csv("data/date_info.csv", parse_dates=["calendar_date"])
store_id_relation = pd.read_csv("data/store_id_relation.csv")

def remove_outliers(data):
    df_0 = data.loc[data.visitors == 0]   
    q1 = np.percentile(data.visitors, 25, axis=0)
    q3 = np.percentile(data.visitors, 75, axis=0)
    k = 2.8
    iqr = q3 - q1
    df_temp = data.loc[data.visitors > q1 - k*iqr]
    df_temp = data.loc[data.visitors < q3 + k*iqr]
    frames = [df_0, df_temp]
    result = pd.concat(frames)
    return result

air["visit_data"] = remove_outliers(air["visit_data"])

df_test = pd.read_csv('sample_submission.csv')
df_test['air_store_id'] = df_test['id'].apply(lambda x: '_'.join(x.split('_')[:2]))
df_test['visit_date'] = df_test['id'].apply(lambda x: x.split('_')[-1])
index_test = df_test['id']
del df_test['id'], df_test['visitors']

In [2]:
print('mapping and dropping useless information in df_hr ...')
s_1 = store_id_relation['air_store_id']
s_2 = store_id_relation['hpg_store_id']
a_h_map = dict(zip(s_2.values, s_1.values))
del s_1, s_2

hpg["reserve"]['air_store_id'] = hpg["reserve"]['hpg_store_id'].map(a_h_map)
hpg["reserve"] = hpg["reserve"].drop('hpg_store_id', axis=1).dropna()


print('mapping and dropping useless information in df_hr Done!')
print("-----------------------------------------------------------------------------------------")

print('mapping and dropping useless information in df_hr ...')

hpg["store_info"]['air_store_id'] = hpg["store_info"]['hpg_store_id'].map(a_h_map)
hpg["store_info"] = hpg["store_info"].drop('hpg_store_id', axis=1).dropna()
print('mapping and dropping useless information in df_hs Done!')
gc.collect()

mapping and dropping useless information in df_hr ...
mapping and dropping useless information in df_hr Done!
-----------------------------------------------------------------------------------------
mapping and dropping useless information in df_hr ...
mapping and dropping useless information in df_hs Done!


53

## Feature Analysis

###### Looking for correlations in data:

# Applying ML

label encoding:

In [3]:
le.fit(air["store_info"]['air_genre_name'])
air["store_info"]['air_genre_name'] = le.fit_transform(air["store_info"]['air_genre_name'])

le.fit(air["store_info"]['air_area_name'])
air["store_info"]['air_area_name'] = le.fit_transform(air["store_info"]['air_area_name'])

le.fit(hpg["store_info"]['hpg_genre_name'])
hpg["store_info"]['hpg_genre_name'] = le.fit_transform(hpg["store_info"]['hpg_genre_name'])

le.fit(hpg["store_info"]['hpg_area_name'])
hpg["store_info"]['hpg_area_name'] = le.fit_transform(hpg["store_info"]['hpg_area_name'])



le.fit(air["store_info"]['air_store_id'])


air["reserve"]['air_store_id'] = le.transform(air["reserve"]['air_store_id'])
air["store_info"]['air_store_id'] = le.transform(air["store_info"]['air_store_id'])
air["visit_data"]['air_store_id'] = le.transform(air["visit_data"]['air_store_id'])
hpg["reserve"]['air_store_id'] = le.transform(hpg["reserve"]['air_store_id'])
hpg["store_info"]['air_store_id'] = le.transform(hpg["store_info"]['air_store_id'])

df_test['air_store_id'] = le.transform(df_test['air_store_id'])

gc.collect()

0

In [4]:
time_format = "%Y-%m-%d %H:%M:%S"
def seperate_date(data):
    # split date feature in reservation datetime
    data_time = pd.to_datetime(data.reserve_datetime, format=time_format)
    data['Year_re']= data_time.dt.year
    data['Month_re'] = data_time.dt.month
    data['DayOfYear_re'] = data_time.dt.dayofyear
    data['DayOfWeek_re'] = data_time.dt.dayofweek
    data['Hour_re'] = data_time.dt.hour
    return data

seperate_date(air["reserve"])


def seperate_date(data):
    # split date feature in reservation datetime
    data_time = pd.to_datetime(data.reserve_datetime, format=time_format)
    data['Year_re_h']= data_time.dt.year
    data['Month_re_h'] = data_time.dt.month
    data['DayOfYear_re_h'] = data_time.dt.dayofyear
    data['DayOfWeek_re_h'] = data_time.dt.dayofweek
    data['Hour_re_h'] = data_time.dt.hour
    return data

seperate_date(hpg["reserve"])


time_format = "%Y-%m-%d %H:%M:%S"
def seperate_date(data):
    # split date feature in reserved visiting datetime
    data_time = pd.to_datetime(data.visit_datetime, format=time_format)
    data['Year_re_visit']= data_time.dt.year
    data['Month_re_visit'] = data_time.dt.month
    data['DayOfYear_re_visit'] = data_time.dt.dayofyear
    data['DayOfWeek_re_visit'] = data_time.dt.dayofweek
    data['Hour_re_visit'] = data_time.dt.hour
    return data

seperate_date(air["reserve"])


def seperate_date(data):
    # split date feature in reserved visiting datetime
    data_time = pd.to_datetime(data.visit_datetime, format=time_format)
    data['Year_re_visit_h']= data_time.dt.year
    data['Month_re_visit_h'] = data_time.dt.month
    data['DayOfYear_re_visit_h'] = data_time.dt.dayofyear
    data['WeekOfYear_re_visit_h'] = data_time.dt.week
    data['DayOfWeek_re_visit_h'] = data_time.dt.dayofweek
    data['Hour_re_visit_h'] = data_time.dt.hour
    return data

seperate_date(hpg["reserve"])

print('seperating date time features done! ...')
gc.collect()

seperating date time features done! ...


135

Merge dataset

In [5]:
features_to_drop = [
        'air_store_id__'
        ]

def merge_df(data, data_to_join):
    # merge dataframes        
    data = data.join(data_to_join, on='air_store_id', rsuffix='__', how='left')   
    return data

def fix_data(data):
    # drop __ data    
    for feature in features_to_drop:
        data.drop(feature, axis=1)
    return data

# Merge to df_train
print('merging dataframes ...')
df_train = merge_df(air["visit_data"], air["reserve"])
df_train = merge_df(df_train, air["store_info"])

hpg["reserve"]['reserve_visitors_hr'] = hpg["reserve"]['reserve_visitors'] 
hpg["reserve"].drop('reserve_visitors', axis=1) 

hpg["store_info"]['latitude_hr'] = hpg["store_info"]['latitude'] 
hpg["store_info"].drop('latitude', axis=1)

hpg["store_info"]['longitude_hr'] = hpg["store_info"]['longitude'] 
hpg["store_info"].drop('longitude', axis=1) 

df_train = merge_df(df_train, hpg["store_info"])
df_train = merge_df(df_train, hpg["store_info"])
gc.collect()
fix_data(df_train)

# Merge to df_test

df_test = merge_df(df_test, air["reserve"])
df_test = merge_df(df_test, air["store_info"])

df_test = merge_df(df_test, hpg["store_info"])
df_test = merge_df(df_test, hpg["reserve"])
gc.collect()
fix_data(df_test)


print('merging dataframes done!')
gc.collect()
print("=========================================================================================")

merging dataframes ...
merging dataframes done!


drop date-time-hour info

In [6]:
def drop_datetime_info(data):
    data = data.drop(['visit_date', 'visit_datetime', 'reserve_datetime', 'visit_datetime__', 'reserve_datetime__'], axis=1)
    return data
df_train = drop_datetime_info(df_train)
df_test = drop_datetime_info(df_test)

In [7]:
train = df_train.fillna(-1)
test = df_test.fillna(-1)

shuffle dataset

In [8]:
from sklearn.utils import shuffle
train = shuffle(train, random_state=524)

X_pretrain, X_prevalid = train_test_split(train, test_size=0.05, random_state=1224, shuffle=False)

X_train = X_pretrain.drop(['visitors'], axis=1)
y_train = np.log1p(X_pretrain["visitors"].values)

X_valid = X_prevalid.drop(['visitors'], axis=1)
y_valid = np.log1p(X_prevalid['visitors'].values)

checking mean rmse

Test run without CV

In [9]:
print("Training LGBM model with cross-validation...")
params = {
    "application": "regression",
    "boosting": "gbdt",
    "learning_rate": 0.001,
    "num_leaves": 32,
    "min_sum_hessian_in_leaf": 1e-2,
    "min_gain_to_split": 0,
    
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8,
    "num_threads": 4,
    "metric": "rmse"
}

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid)

watchlist = [d_train, d_valid]

# default value of folds: 5
lgb_model1 = lgb.train(params, train_set=d_train, num_boost_round=37000, valid_sets=watchlist, verbose_eval=1000)

print("Model trained. Predicting...")

test_probs = lgb_model1.predict(test)
test_probs = np.expm1(test_probs)

result = pd.DataFrame({"id": index_test, "visitors": test_probs})
result.to_csv("LGB_sub.csv", index=False) # 0.60
print("Prediction complete.")

Training LGBM model with cross-validation...
[100]	training's rmse: 0.791755	valid_1's rmse: 0.783862
[200]	training's rmse: 0.784701	valid_1's rmse: 0.776878
[300]	training's rmse: 0.778103	valid_1's rmse: 0.770322
[400]	training's rmse: 0.772073	valid_1's rmse: 0.764378
[500]	training's rmse: 0.766316	valid_1's rmse: 0.758666
[600]	training's rmse: 0.760942	valid_1's rmse: 0.753373
[700]	training's rmse: 0.755944	valid_1's rmse: 0.748463
[800]	training's rmse: 0.750981	valid_1's rmse: 0.743616
[900]	training's rmse: 0.746306	valid_1's rmse: 0.738996
[1000]	training's rmse: 0.741833	valid_1's rmse: 0.734585
[1100]	training's rmse: 0.737489	valid_1's rmse: 0.730306
[1200]	training's rmse: 0.73338	valid_1's rmse: 0.72616
[1300]	training's rmse: 0.729542	valid_1's rmse: 0.722319
[1400]	training's rmse: 0.725863	valid_1's rmse: 0.71868
[1500]	training's rmse: 0.722139	valid_1's rmse: 0.715002
[1600]	training's rmse: 0.718587	valid_1's rmse: 0.71147
[1700]	training's rmse: 0.715217	valid_1

[14200]	training's rmse: 0.603749	valid_1's rmse: 0.601135
[14300]	training's rmse: 0.603554	valid_1's rmse: 0.60096
[14400]	training's rmse: 0.603369	valid_1's rmse: 0.600796
[14500]	training's rmse: 0.603186	valid_1's rmse: 0.600629
[14600]	training's rmse: 0.60301	valid_1's rmse: 0.600461
[14700]	training's rmse: 0.602822	valid_1's rmse: 0.600285
[14800]	training's rmse: 0.602636	valid_1's rmse: 0.600111
[14900]	training's rmse: 0.60246	valid_1's rmse: 0.599942
[15000]	training's rmse: 0.602293	valid_1's rmse: 0.599785
[15100]	training's rmse: 0.602128	valid_1's rmse: 0.599637
[15200]	training's rmse: 0.601975	valid_1's rmse: 0.599499
[15300]	training's rmse: 0.601825	valid_1's rmse: 0.59936
[15400]	training's rmse: 0.601657	valid_1's rmse: 0.599205
[15500]	training's rmse: 0.601512	valid_1's rmse: 0.599073
[15600]	training's rmse: 0.601357	valid_1's rmse: 0.598936
[15700]	training's rmse: 0.601206	valid_1's rmse: 0.598803
[15800]	training's rmse: 0.60107	valid_1's rmse: 0.598679
[1

[28200]	training's rmse: 0.594617	valid_1's rmse: 0.593466
[28300]	training's rmse: 0.594604	valid_1's rmse: 0.59346
[28400]	training's rmse: 0.59459	valid_1's rmse: 0.593455
[28500]	training's rmse: 0.594579	valid_1's rmse: 0.59345
[28600]	training's rmse: 0.594568	valid_1's rmse: 0.593443
[28700]	training's rmse: 0.594555	valid_1's rmse: 0.593434
[28800]	training's rmse: 0.594541	valid_1's rmse: 0.593427
[28900]	training's rmse: 0.594528	valid_1's rmse: 0.593422
[29000]	training's rmse: 0.594515	valid_1's rmse: 0.593414
[29100]	training's rmse: 0.594502	valid_1's rmse: 0.593409
[29200]	training's rmse: 0.594491	valid_1's rmse: 0.593405
[29300]	training's rmse: 0.59448	valid_1's rmse: 0.593399
[29400]	training's rmse: 0.594468	valid_1's rmse: 0.593391
[29500]	training's rmse: 0.594456	valid_1's rmse: 0.593383
[29600]	training's rmse: 0.594444	valid_1's rmse: 0.593373
[29700]	training's rmse: 0.594432	valid_1's rmse: 0.593364
[29800]	training's rmse: 0.594421	valid_1's rmse: 0.593358
[